# Laboratorio de regresión - 4

|                |   |
:----------------|---|
| **Nombre**     | Ernesto Andrés González Lomelí  |
| **Fecha**      | 06/02/2024  |
| **Expediente** | 746843  |

## Modelos penalizados

Hasta ahora la función de costo que usamos para decidir qué tan bueno es nuestro modelo al momento de ajustar es:

$$ \text{RSS} = \sum_{i=1}^n e_i^2 = \sum_{i=1}^n (y_i - \hat{y_i})^2 $$

Dado que los errores obtenidos son una combinación de sesgo y varianza, puede ser que se sesgue un parámetro para minimizar el error. Esto significa que el modelo puede decidir que la salida no sea una combinación de los factores, sino una fuerte predilección sobre uno de los factores solamente. 

E.g. se quiere ajustar un modelo

$$ \hat{z} = \hat{\beta_0} + \hat{\beta_1} x + \hat{\beta_2} y $$

Se ajusta el modelo y se decide que la mejor decisión es $\hat{\beta_1} = 10000$ y $\hat{\beta_2}=50$. Considera limitaciones de problemas reales:
- Quizás los parámetros son ajustes de maquinaria que se deben realizar para conseguir el mejor producto posible, y que $10000$ sea imposible de asignar.
- Quizás los datos actuales están sesgados y sólo hacen parecer que uno de los factores importa más que el otro.

Una de las formas en las que se puede mitigar este problema es penalizando a los parámetros del modelo, cambiando la función de costo:

$$ \text{RSS}_{L2} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p \hat{\beta_j}^2 $$

El *L2* significa que se está agregando una penalización de segundo orden. Lo que hace esta penalización es que los factores ahora sólo tendrán permitido crecer si hay una reducción al menos proporcional en el error (sacrificamos sesgo, pero reducimos la varianza).

Asimismo, existe la penalización *L1*

$$ \text{RSS}_{L1} = \sum_{i=1}^n e_i^2  + \lambda \sum_{j=1}^p |\hat{\beta_j}| $$

A las penalizaciones *L2* y *L1* se les conoce también como Ridge y Lasso, respectivamente.

Para realizar una regresión con penalización de Ridge o de Lasso usamos el objeto `Ridge(alpha=?)` o `Lasso(alpha=?)` en lugar de `LinearRegression()` de `sklearn`.

Utiliza el dataset de publicidad (Advertising.csv) y realiza 3 regresiones múltiples:

$$ \text{sales} = \beta_0 + \beta_1 (\text{TV}) + \beta_2 (\text{radio}) + \beta_3 (\text{newspaper}) + \epsilon $$

1. Sin penalización
2. Con penalización L2
3. Con penalización L1

Compara los resultados de los parámetros y sus *p-values*, y los $R^2$ resultantes.

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [2]:
data=pd.read_csv('Advertising.csv')
data

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9
...,...,...,...,...,...
195,196,38.2,3.7,13.8,7.6
196,197,94.2,4.9,8.1,9.7
197,198,177.0,9.3,6.4,12.8
198,199,283.6,42.0,66.2,25.5


In [3]:
X = data[['TV', 'radio', 'newspaper']]
y = data['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

LINEAL SIN PENALIZACIÓN

In [4]:
linear_model = LinearRegression()
linear_model.fit(X_train,y_train)

LinearRegression()

In [9]:
#B0
linear_model.intercept_

2.70894909251591

In [14]:
#B1, B2...
linear_model.coef_

array([0.04405928, 0.1992875 , 0.00688245])

In [15]:
y_pred = linear_model.predict(X_test)

In [16]:
#coeficiente de determinación (R^2)
r2 = r2_score(y_test, y_pred)
r2

0.8609466508230368

In [51]:
import statsmodels.api as sm

In [52]:
x_con_constante=sm.add_constant(X_train)
ols=sm.OLS(y_train, x_con_constante)
ols_results=ols.fit()
ols_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  sales   R-squared:                       0.906
Model:                            OLS   Adj. R-squared:                  0.903
Method:                 Least Squares   F-statistic:                     434.5
Date:                Wed, 12 Feb 2025   Prob (F-statistic):           1.88e-69
Time:                        14:48:24   Log-Likelihood:                -262.21
No. Observations:                 140   AIC:                             532.4
Df Residuals:                     136   BIC:                             544.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.7089      0.374      7.250      0.000       1.970       3.448
TV             0.0441      0.002     27.219      0.000       0.041       0.047
radio          0.1993      0.010     20.195      0.000       0.180       0.219
newspaper      0.0069      0.007      0.988      0.325      -0.007       0.021
==============================================================================
Omnibus:                       68.437   Durbin-Watson:                   2.285
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              325.342
Skew:                          -1.709   Prob(JB):                     2.25e-71
Kurtosis:                       9.640   Cond. No.                         500.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

La regresión lineal estándar tiene un coeficiente de determinación de 0.8609, lo que nos dice que el modelo explica el 86.09% de la variabilidad de las ventas. Los coeficientes muestran que radio tiene la mayor influencia en las ventas, seguido por TV, mientras que newspaper tiene muy poco impacto.

TV y Radio son altamente significativas (p-values casi cero), lo que confirma su relación fuerte con las ventas.
Newspaper no es significativa, lo que nos dice que no contribuye de manera significativa a predecir las ventas.

**Penalización L1 con LASSO**

In [73]:
lasso_model = Lasso(alpha=1)
lasso_model.fit(X_train, y_train)

Lasso(alpha=1)

In [74]:
#B0
lasso_model.intercept_

2.8583889551720087

In [75]:
#B1,B2,B3
lasso_model.coef_

array([0.04397345, 0.19528096, 0.00551083])

In [76]:
lasso_y_pred = lasso_model.predict(X_test)

In [77]:
lasso_r2 = r2_score(y_test, lasso_y_pred)
lasso_r2

0.8651496069722558

In [78]:
lasso_const = sm.add_constant(X_train)
lasso_ols_model = sm.OLS(y_train, lasso_const).fit()
lasso_pvalues = lasso_ols_model.pvalues


In [79]:
lasso_ols_summary=pd.DataFrame({
    "Variable": ["Intercept", "TV", "Radio", "Newspaper"],
    "Lasso p-value": lasso_ols_model.pvalues.values})
lasso_ols_summary

,Variable,Lasso p-value
0,Intercept,2.848109e-11
1,TV,6.788687e-57
2,Radio,9.192647e-43
3,Newspaper,3.249067e-01


p-values permanecen iguales, lo que dice que las mismas variables mantienen su significancia estadística

Aquí, newspaper mantiene un pequeño coeficiente, lo que indica que la penalización no fue lo suficientemente fuerte para descartarla por completo.

**PENALIZACION L2 CON RIDGE**

In [88]:
ridge_model = Ridge(alpha=10.0)
ridge_model.fit(X_train, y_train)

Ridge(alpha=10.0)

In [89]:
#B0
ridge_model.intercept_

2.710158542833552

In [90]:
#B1,B2,B3
ridge_model.coef_

array([0.04405956, 0.19921229, 0.00689942])

In [91]:
ridge_y_pred = ridge_model.predict(X_test)

In [92]:
ridge_r2 = r2_score(y_test, ridge_y_pred)
ridge_r2

0.8609851905762931

In [93]:
ridge_X_with_const = sm.add_constant(X_train)
ridge_ols_model = sm.OLS(y_train, ridge_X_with_const).fit()

In [94]:
ridge_ols_summary=pd.DataFrame({
    "Variable": ["Intercept", "TV", "Radio", "Newspaper"],
    "ridge p-value": ridge_ols_model.pvalues.values})
ridge_ols_summary

,Variable,ridge p-value
0,Intercept,2.848109e-11
1,TV,6.788687e-57
2,Radio,9.192647e-43
3,Newspaper,3.249067e-01


Aunque el p-value no cambia, los coeficientes se reducieron para controlar su magnitud y evitar posibles problemas de sobreajuste.
Esto nos dice que Ridge es útil para estabilizar el modelo y obtener coeficientes más pequeños y manejables, pero no elimina variables irrelevantes.